## This is the updated system
1. Fetch data from NSE
2. Automatically generate prices and stop loss
3. 

## Instructions
1. See instructions for each cell
2. You need to change only the PARAMETERS, nothing else
3. You need to have 2 bhav copy files, one to generate orders and the next to evaluate performance
4. Based on the previous day orders, **results.xlsx** file is generated. You need to fill in the current day's open price and run the cells below it to generate orders. This would generate **orders.csv** file
5. At the end of the day, you need to download the bhav copy results to evaluate performance. This would generate a **peformance.csv** file

Save all these files at the end of the day in a new folder. We would automate this process also soon

In [1]:
# Import libraries
import pandas as pd
import os

In [2]:
%load_ext autoreload

In [5]:
%autoreload
from utils import *
fetch_preopen_data()

,SYMBOL,PctChange,OpenPrice
0,COALINDIA,1.14,282.80
1,GAIL,1.13,381.95
2,INFRATEL,-0.16,275.25
3,NTPC,-0.09,169.05
4,HINDPETRO,-0.02,246.60
5,AXISBANK,-0.02,651.65
6,LUPIN,0.98,953.65
7,SBIN,0.88,287.35
8,SUNPHARMA,0.78,643.00
9,TECHM,0.75,772.00


## Parameters that could be changed

In [2]:
UNIVERSE = 'NIFTY50' # Universe to be searched
STOP_LOSS = 3 # Stop loss for the order
NUM_STOCKS = 5 # Number of stocks to sell
CAPITAL = 20000
LEVERAGE = 1

In [12]:
### Don't change the below cell

In [3]:
orders = pd.read_csv(ORDER_FILENAME, parse_dates=['TIMESTAMP'],
                    usecols=range(13))
symbols = pd.read_excel('universe.xlsx', sheet_name=UNIVERSE, header=None).values.ravel()
df = orders[orders['SYMBOL'].isin(symbols)]
df = df[df['SERIES'] == "EQ"].reset_index(drop=True)
df['RET'] = (df['CLOSE']/df['PREVCLOSE']) - 1
result = df.sort_values(by='RET', ascending = False).iloc[:NUM_STOCKS]
result['OPENPRICE'] = 0
result[['TIMESTAMP', 'SYMBOL', 'OPENPRICE']].to_excel('output.xlsx', index=False)
print('Update OPENPRICE in the output.xlsx file')
result[['TIMESTAMP', 'SYMBOL', 'OPENPRICE']]

FileNotFoundError: [Errno 2] No such file or directory: 'cm29AUG2018bhav.csv.zip'

### Update OPEN PRICE in output.xlsx file and run the below cell

In [14]:
trading_capital = CAPITAL * LEVERAGE
orders = pd.read_excel('output.xlsx')
num_stock = len(orders)
orders['price'] = orders['OPENPRICE']
orders['stop_loss'] = (orders['price'] * 1.03).round(2)
orders['qty'] = (trading_capital/num_stock/orders['price']).round()
orders.to_csv('orders.csv', index=False)
print('Orders file created')
orders

Orders file created


,TIMESTAMP,SYMBOL,OPENPRICE,price,stop_loss,qty
0,2018-08-24,VEDL,225.95,225.95,232.73,18.0
1,2018-08-24,ONGC,175.45,175.45,180.71,23.0
2,2018-08-24,GRASIM,1063.10,1063.10,1094.99,4.0
3,2018-08-24,AXISBANK,643.00,643.00,662.29,6.0
4,2018-08-24,ZEEL,517.00,517.00,532.51,8.0


In [16]:
### Run the below cell to evaluate peformance

In [18]:
def isPrice(price, high, low):
    if price >= low and price <=high:
        return True
    else:
        False

orders = pd.read_csv('orders.csv', index_col='TIMESTAMP', parse_dates=True)
end_of_day = pd.read_csv(RESULT_FILENAME, parse_dates=['TIMESTAMP'],
                    usecols=range(13))
end_of_day = end_of_day[end_of_day['SERIES'] == "EQ"]
orders = orders.merge(end_of_day)
orders['SELL'] = [price if isPrice(price, high, low) else close for
                 price, high, low, close in 
                 zip(orders['price'], orders['HIGH'], orders['LOW'], orders[REF_PRICE])]
orders['BUY'] = [price if isPrice(price, high, low) else close for
                 price, high, low, close in 
                 zip(orders['stop_loss'], orders['HIGH'], orders['LOW'], orders[REF_PRICE])]
orders['profit_per_unit'] = orders['SELL'] - orders['BUY']
orders['total_profit'] = orders['profit_per_unit'] * orders['qty']
cols = ['TIMESTAMP', 'SYMBOL', 'BUY', 'SELL', 'profit_per_unit', 'total_profit']
orders.to_csv('performance.csv', index=False)
orders[cols]

,TIMESTAMP,SYMBOL,BUY,SELL,profit_per_unit,total_profit
0,2018-08-27,VEDL,225.50,225.95,0.45,8.1
1,2018-08-27,ONGC,176.05,175.45,-0.60,-13.8
2,2018-08-27,GRASIM,1078.45,1063.10,-15.35,-61.4
3,2018-08-27,AXISBANK,650.20,643.00,-7.20,-43.2
4,2018-08-27,ZEEL,511.20,517.00,5.80,46.4
